<a href="https://colab.research.google.com/github/kentaojapi/demucs_colab_gui/blob/main/demucs_colab_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DemucsをGUI操作だけで利用できるようにするnotebookです。

* GitHub<br>
https://github.com/facebookresearch/demucs<br>


# 1. 環境のセットアップ

In [ ]:

#@title 環境のセットアップ
#@markdown このセルを実行することで、必要な環境がセットアップされます。 \
#@markdown セル実行は左の再生ボタンのようなマークを押して下さい。
!nvidia-smi
%cd /content
!pip install demucs
!pip install yt-dlp moviepy
!pip install imageio==2.4.1


!mkdir -p /content/audios
!mkdir -p /content/confirming


import os
import shutil
import IPython.display as ipd
import wave

from google.colab import files
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from yt_dlp import YoutubeDL


def delete_files_in_directory(directory_path: str) -> None:
    if not os.path.exists(directory_path):
        return
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)


class AudioClipper:
    DESTINATION_PATH: str = '/content/confirming'
    START_TIME: int = 0
    END_TIME: int = 30

    def __init__(self, input_path: str) -> None:
        self.input_path: str = input_path

    @property
    def output_path(self) -> str:
        file_name: str = os.path.basename(self.input_path)
        return os.path.join(self.DESTINATION_PATH, file_name)

    def run(self) -> None:
        with wave.open(self.input_path, 'rb') as input_wav:
            sample_width = input_wav.getsampwidth()
            num_channels = input_wav.getnchannels()
            frame_rate = input_wav.getframerate()
            frames_to_clip = int(frame_rate * self.END_TIME)
            input_wav.setpos(int(frame_rate * self.START_TIME))
            frames = input_wav.readframes(frames_to_clip)

        with wave.open(self.output_path, 'wb') as output_wav:
            output_wav.setnchannels(num_channels)
            output_wav.setsampwidth(sample_width)
            output_wav.setframerate(frame_rate)
            output_wav.writeframes(frames)


input_audio_path = '/content/audios/input_clip.wav'


# 2. 対象音源の選択

対象の音源は、以下2つの方法で指定することが可能です。  
**必ずaかbのどちらかのみを実行するようにして下さい。**

* a. Youtube動画  
* b. WAVファイル

In [ ]:
#@title a. Youtube動画
#@markdown 動画のURLを入力してください。
video_url = 'https://www.youtube.com/watch?v=2fDzCWNS3ig' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を入力してください。その後セルを実行してください。
start_sec =  0#@param {type:"integer"}
end_sec =  233#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

!mkdir -p /content/videos

# 動画ダウンロード
download_resolution = 360
full_video_path = '/content/videos/full_video.mp4'
input_clip_path = '/content/videos/input_clip.mp4'
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    if start_sec > video.duration:
        raise ValueError(f"start_sec（秒）が動画の長さを超えた値で指定されています。動画の長さは{video.duration}秒です。")
    if end_sec > video.duration:
        raise ValueError(f"end_sec（秒）が動画の長さを超えた値で指定されています。動画の長さは{video.duration}秒です。")
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

# 音声抽出
clip = VideoFileClip(input_clip_path)
clip.audio.write_audiofile(input_audio_path, codec='pcm_s16le')


In [ ]:
#@title b. WAVファイル
#@markdown このセルを実行後にアップロードボタンが表示されます。対象のWAVファイルをアップロードしてください。 \
#@markdown ファイルのアップロードにはしばらく時間を要します。
input_raw_audio_path = '/content/audios/input_raw.wav'

for filename, filedata in files.upload().items():
    shutil.move(filename, input_raw_audio_path)

In [ ]:
#@title b.続き
#@markdown アップロードしたWAVファイルの切り抜き範囲(秒)を入力してください。その後セルを実行してください。
from google.colab import files
import IPython.display as ipd
import ipywidgets as widgets
import numpy as np
import librosa
import soundfile as sf

data, sr = librosa.load(input_raw_audio_path, sr=None)
duration = len(data) / sr

# トリミングする範囲を指定（開始秒数と終了秒数）
start_sec = 0#@param {type:"integer"}
end_sec = 200#@param {type:"integer"}

if start_sec > duration:
    raise ValueError(f"start_sec（秒）が動画の長さを超えた値で指定されています。音源の長さは{duration}秒です。")
if end_sec > duration:
    raise ValueError(f"end_sec（秒）が動画の長さを超えた値で指定されています。音源の長さは{duration}秒です。")

# 開始フレームと終了フレームを計算
start_frame = int(start_sec * sr)
end_frame = int(end_sec * sr)

# トリミング
trimmed_data = data[start_frame:end_frame]

# トリミングされたwavファイルを保存
sf.write(input_audio_path, trimmed_data, sr)

In [ ]:
#@title 音源を確認（a. b. 共通）
#@markdown　このセルを実行すると指定した音源の対象範囲を試聴することができます。 \
#@markdown　再生秒数は冒頭30秒に制限しています。（ランタイムが落ちやすいため。） \
#@markdown　また、実行にはしばらく時間を要します。
audio_clipper = AudioClipper(input_audio_path)
audio_clipper.run()
ipd.Audio(audio_clipper.output_path, rate=44100)

# 3. 音源分離の実行


In [ ]:
#@title 音源分離の実行
#@markdown プルダウンから分離したい対象のパートを選択した上でセルを実行して下さい。　\
#@markdown　音源分離処理が実行されます。

input_directory_path = '/content/separated/htdemucs/input_clip/'
delete_files_in_directory(input_directory_path)

extraction_source = "vocals" #@param ["vocals", "drums", "bass", "other"]
!python3 -m demucs --two-stems={extraction_source} {input_audio_path}

In [ ]:
#@title 音源を確認
#@markdown　このセルを実行すると分離した対象を試聴することができます。 \
#@markdown　再生秒数は冒頭30秒に制限しています。（ランタイムが落ちやすいため。） \
#@markdown　また、実行にはしばらく時間を要します。
separated_source_wav: str = f"/content/separated/htdemucs/input_clip/{extraction_source}.wav"
audio_clipper = AudioClipper(separated_source_wav)
audio_clipper.run()
ipd.Audio(audio_clipper.output_path, rate=44100)

# 4. ダウンロード

In [ ]:
#@title 分離した音源のダウンロード
#@markdown セルを実行することでzipファイル形式でダウンロードできます。 \
#@markdown zipファイルには以下のファイルが含まれます。
#@markdown 1. 抽出対象パートのWAVファイル
#@markdown 2. 抽出対象パートのみを楽曲から取り除いたトラックのWAVファイル \
#@markdown
#@markdown zipファイルのダウンロードには時間を要します。

output_directory_path = '/content/separated/zip/'
delete_files_in_directory(output_directory_path)

directory_path = '/content/separated/htdemucs/input_clip/'
zip_path = '/content/separated/zip/separated_data.zip'
zip_path_without_extension = os.path.splitext(zip_path)[0]
shutil.make_archive(zip_path_without_extension, 'zip', directory_path)


files.download(zip_path)